# Variable Naming
Hypster provides sensible defaults for naming your variables to keep your code **DRY** (**D**on't **R**epeat **Y**ourself)

## Explicit Naming
You can explicitly name your variables using the `name` parameter:

In [18]:
from hypster import HP, config

@config
def explicit_naming(hp: HP):
    var = hp.select(['o1', 'o2'], name='my_explicit_variable')

## Automatic Naming

Hypster automatically infers variable names by utilizing the variable names, dictionary keys, and keyword arguments:

1. Variable Names
   - Example: `a = hp.select(['option1', 'option2'])`
   - Result: 'a' will be the name of this parameter

2. Dictionary Keys
   - Example: `config = {'learning_rate': hp.number_input(0.001)}`
   - Result: The dictionary key 'learning_rate' will be the name of this parameter

3. Class and Function Keyword Arguments
   - Example: `Model(hidden_size=hp.select([64, 128, 256]))`
   - Result: The keyword argument 'hidden_size' will be the name of this parameter

For nested structures, Hypster uses dot notation `(key.nested_key)` to represent the hierarchy. For example:
```python
model = Model(model_type=hp.select(['cnn', 'rnn']), # Automatically named 'model.model_type'
              model_kwargs={'lr' : hp.number_input(0.1)} # Automatically named 'model.model_kwargs.lr'
             )
```

### Example Use-Cases:
1. Variable Assignment

In [19]:
@config
def automatic_naming(hp: HP):
    # This will be automatically named 'var'
    var = hp.select(['o1', 'o2'])
    # This will be automatically named 'model_type'
    model_type = hp.select(['cnn', 'rnn'])

2. Dictionary Keys:

In [20]:
@config
def dict_naming(hp: HP):
    config = {
        'model_type': hp.select(['cnn', 'rnn']),  # Automatically named 'config.model_type'
        'learning_rate': hp.number_input(0.001)   # Automatically named 'config.learning_rate'
    }

3. Class and function Keyword Arguments:

In [23]:
from hypster import HP, config

@config
def class_kwargs_naming(hp: HP):
    # Note new class definitions (or imports) need to be inside the config function
    class ModelConfig:
        def __init__(self, model_type, learning_rate):
            self.model_type = model_type
            self.learning_rate = learning_rate
    
    def func(param):
        return

    model = ModelConfig(model_type=hp.select(['cnn', 'rnn']), # Automatically named 'model.model_type'
                        learning_rate=hp.number_input(0.001)  # Automatically named 'model.learning_rate'
                        )
    
    var = func(param=hp.select(['option1', 'option2']))  # Automatically named 'var.param'

In [25]:
results = class_kwargs_naming(selections={'model.model_type': 'cnn',
                                          'var.param': 'option1'})
print(results["model"].model_type)
print(results["model"].learning_rate)

cnn
0.001
